### Попытаться ответить на вопросы/выдвинуть гипотезы

##### 3 товара с акцией (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)


фильтрация по товарам на которые есть скидки. думаю надо использовать актуальную скидку, хотя в процессе тестирования рек. системы лучше зафиксировать скидки номенклатруно.

##### 1 новый товар (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)


если юзер вообще не покупал никаких товаров можно показывать новый товар из наиболее популярный категорий

##### 1 товар для роста среднего чека (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)

найти наиболее часто покупаемые товары юзером и предложить схожие, но немного дороже. полагаю, что лучше показыват товары дороже на 10-20%, т.к. это будет не так ощутимо финансово. мб увеличнеие прайса в 2 раза отпугнуть от покупки.

### Поэкспериментировать с ALS (grid-search)

In [1172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import metrics

from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [1194]:
data = pd.read_csv('transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [1195]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [1196]:
data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))
data_train['price'].max()

<ipython-input-1196-73e31537f690>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))


499.99

In [1197]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    popularity['share_unique_users'] = popularity['share_unique_users'] / data['user_id'].nunique()
    
    top_popular = popularity[popularity['share_unique_users'] > 0.3].item_id.tolist()
         #слишком много таких товаров -> слишком сильно падает пресижн
    #data.loc[data['item_id'].isin(top_popular), 'item_id'] = 999_999
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    not_top_popular = popularity[popularity['share_unique_users'] == 0.0004].item_id.tolist()
    data.loc[data['item_id'].isin(not_top_popular), 'item_id'] = 999_999
        
        
    # Уберем товары, которые не продавались за последние 12 месяцев 
        #слишком много таких товаров -> слишком сильно падает пресижн
    #not_sold = data_train[data_train['week_no'] > 68].item_id.tolist()
    #data.loc[data['item_id'].isin(not_sold), 'item_id'] = 999_999
    
    # Уберем не интересные для рекоммендаций категории (department)
    id_department = item_features.groupby(['item_id', 'department']).nunique().reset_index()
    not_top_department = item_features['department'].value_counts()[10:].index.tolist()
    id_department = id_department[id_department['department'].isin(not_top_department)]
    data.loc[data['item_id'].isin(id_department['item_id']), 'item_id'] = 999_999
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    cheap = data_train[data_train['price'] < data_train['price'].quantile(.05)].item_id.tolist()
         #слишком много таких товаров -> слишком сильно падает пресижн
    #data.loc[data['item_id'].isin(cheap), 'item_id'] = 999_999
    
    # Уберем слишком дорогие товары
    
    expensive = data_train[data_train['price'] > data_train['price'].quantile(.999)].item_id.tolist()
    data.loc[data['item_id'].isin(expensive), 'item_id'] = 999_999
    
    return data
    
def postfilter_items(user_id, recommednations):
    pass

In [1198]:
data_train = prefilter_items(data_train)

G:\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
G:\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [1199]:
train_result = data_train.groupby('user_id')['item_id'].unique().reset_index()
train_result.columns=['user_id', 'actual']

In [1200]:
test_result = data_test.groupby('user_id')['item_id'].unique().reset_index()
test_result.columns=['user_id', 'actual']

In [1201]:
result = test_result

In [1202]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [1203]:
#data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

In [1204]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

item_id,25671,26691,27021,27159,27479,27658,27745,27754,27812,27951,...,18056453,18104617,18104908,18105225,18105264,18106229,18106286,18119004,18119016,18120821
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1205]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [1206]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999_999]], 
                                    recalculate_user=True)]
    return res

In [1207]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [1216]:
#подбирала парметры на топ-5000, т.к. экономнее по времени

In [1112]:
import time

params = {
    'factors':[10, 50, 100, 150],
    'iterations':[5, 10, 15, 20, 25],
    'regularization':[0.001, 0.01, 0.05, 0.1]
}

result_feat = {
    'factors':[],
    'iterations':[],
    'regularization':[],
    'precision_train':[],
    'precision_test':[]
}

start_time = time.time()
i = 0
for factor in params['factors']:
    for iteration in params['iterations']:
        for regular in params['regularization']: 
                result_feat['factors'].append(factor)
                result_feat['iterations'].append(iteration)
                result_feat['regularization'].append(regular)
                model = AlternatingLeastSquares(factors=factor,
                                                regularization=regular,
                                                iterations=iteration, 
                                                calculate_training_loss=True, 
                                                num_threads=10,
                                                use_gpu=False)        
                model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
                
                res_name = f'ALS: factors: {factor}, iterations: {iteration}, regularization: {regular}'
                train_result[res_name] = train_result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
                precision_train = train_result.apply(lambda row: precision_at_k(row[res_name], row['actual']), axis=1).mean()
        
                test_result[res_name] = test_result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
                precision_test = test_result.apply(lambda row: precision_at_k(row[res_name], row['actual']), axis=1).mean()
                
                result_feat['precision_train'].append(precision_train)
                result_feat['precision_test'].append(precision_test)
                
                i +=1
                total_time = time.time()
                
                if i % 10 == 0 or i == 1:                
                    print(f'Итерация {i} из 80. Времени затрачено: {((total_time - start_time) / 60):.2f} минут.')

Итерация 1 из 80. Времени затрачено: 0.43 минут.
Итерация 10 из 80. Времени затрачено: 4.31 минут.
Итерация 20 из 80. Времени затрачено: 8.73 минут.
Итерация 30 из 80. Времени затрачено: 15.28 минут.
Итерация 40 из 80. Времени затрачено: 21.75 минут.
Итерация 50 из 80. Времени затрачено: 32.67 минут.
Итерация 60 из 80. Времени затрачено: 44.56 минут.
Итерация 70 из 80. Времени затрачено: 62.73 минут.
Итерация 80 из 80. Времени затрачено: 80.47 минут.


In [1145]:
param_result = pd.DataFrame(result_feat)
param_result.sort_values(by=['precision_test'], ascending=False).head(20)

,factors,iterations,regularization,precision_train,precision_test
2,10,5,0.050,0.63808,0.190758
19,10,25,0.100,0.63920,0.189453
10,10,15,0.050,0.63944,0.188749
7,10,10,0.100,0.63720,0.188147
6,10,10,0.050,0.63760,0.188046
1,10,5,0.010,0.63704,0.187946
0,10,5,0.001,0.64288,0.187745
11,10,15,0.100,0.63984,0.187243
9,10,15,0.010,0.64080,0.187243
3,10,5,0.100,0.64312,0.187142


In [1219]:
#для обучения в итоге использовала полный датасет после обработки (без 999999). На топ-5000 пресижн был ниже после тфидф и бм25.

#взяла параметры с наиболее лучшим пресиджном по трейну и тесту (100, 5, 0.1)

In [1208]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.1,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=10,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/5 [00:00<?, ?it/s]

Wall time: 17min 48s


In [1209]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.1864389753892494

In [1210]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [1211]:
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [1212]:
result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

0.20793571069813838

In [1213]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [1214]:
result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

0.24640883977900263

In [1215]:
result.head(2)

,user_id,actual,als,als_tfidf,als_bm25
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1056509, 1100972, 990797, 866211, 1077555]","[1082185, 995242, 866211, 1082212, 981760]","[1082185, 995242, 1024306, 1075074, 962568]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1092026, 1022003, 951590, 8090521, 1106523]","[1106523, 951590, 1133018, 1082185, 883404]","[883404, 1044078, 951590, 1098066, 826249]"


Conclusions:

Дополнительно обработала массив (prefilter_items). Не стала убирать очень популярные товары, дешевые и длительно хранящиеся, т.к. тогда сильно падал пресижн. 
В целом, т.к. мы сравниваем с уже купленными товарами, то логчино, что там много дешевых/популярных товаров. Вероятно если сделать эти обработки а запустить на тест на недельку - он бы показал хороший результат. 
Но на таких данных решила оставить так.

При выборе параметров для обечения, основывалась на хороших показателях на трейне и тесте, т.к. думаю 60% на трейне недообучена модель.
По факту лучше показал именно тот вариант,что я выбрала.

Также подбирала парметры на топ5000, однако обучать решила на полном обработанном датасете, т.к. я его итак значительно уменьшила обработкой.